In [2]:
import matplotlib.pyplot as plt

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline
import os
from utils import *

import os

# import package from root directory
import sys
sys.path.append('/home/cat/code/bmi/utils/calcium/')
import calcium


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
root_dir = '/media/cat/4TB/donato/andres/cohort9/DON-014618/'
sub_dir = 'data/suite2p/plane0/'


c_may2 = calcium.Calcium()

session = '20230502'
c_may2.data_dir = os.path.join(root_dir, 
                          session,
                          sub_dir)

c_may2.load_suite2p()

# same thing but for may 3rd
c_may3 = calcium.Calcium()
session = '20230503'
c_may3.data_dir = os.path.join(root_dir,
                                 session,
                                    sub_dir)
c_may3.load_suite2p()


#


In [23]:
print (len(c.F))

532


In [63]:

# plot contour


d = np.load('/media/cat/4TB/donato/andres/cohort9/DON-014618/day0/rois_pixels_and_thresholds_day0.npz',
            allow_pickle=True)

#
cell_ids = d['cell_ids']
contours_all_cells = d['contours_all_cells']

#
clrs=['blue','lightblue','red','pink']

plt.figure()

plt.imshow(img)

for k in range(len(cell_ids)):
    contour = contours_all_cells[k][0]

    #
    plt.plot(contour[:,0],
             contour[:,1],
             c=clrs[k],
             linewidth=10)

#
for k in range(len(c_may2.F)):
    contour = c_may2.get_footprint_contour(k)


    plt.plot(contour[:,0],
             contour[:,1],
             c='blue') 

if False:
    for k in range(len(c_may3.F)):
        contour = c_may3.get_footprint_contour(k)

        plt.plot(contour[:,0],
                contour[:,1],
                c='red')

#
#plt.ylim(0,512)
#plt.xlim(0,512)
plt.show()



In [68]:

d = np.load('/media/cat/4TB/donato/andres/cohort9/DON-014618/day0/rois_pixels_and_thresholds_day0.npz',
            allow_pickle=True)

#
cell_ids = d['cell_ids']
print (cell_ids)
ensemble1_footprints = d['ensemble1_footprints']
print (ensemble1_footprints.shape)
ensemble2_footprints = d['ensemble2_footprints']
print (ensemble2_footprints.shape)


#print (ensemble1_footprints[0])
# show footprints as images
img = np.zeros((512,512),'float32')+np.nan
for p in range(2):
    for k in range(ensemble1_footprints[p].shape[0]):
        img[ensemble1_footprints[p][k][0],
            ensemble1_footprints[p][k][1]]=1
# same but for ensemble 2
for p in range(2):
    for k in range(ensemble2_footprints[p].shape[0]):
        img[ensemble2_footprints[p][k][0],
            ensemble2_footprints[p][k][1]]=1


plt.figure()
plt.imshow(img)
plt.xlim(0,512)
plt.ylim(0,512)

plt.show()

[ 5  4 49 32]
(2,)
(2,)


In [67]:
c_may2.load_footprints()
print (c_may2.footprints.shape)

(532, 512, 512)


In [38]:
####################################################
################## LOAD DATA #######################
####################################################


#root_dir = r'Z:\RG Donato\Microscopy\Andres\BMI Project\cohort_7_sessions'
root_dir = '/media/cat/4TB/donato/andres/cohort8'

animal_id = 'DON-014265'
#animal_id = 'DON-012503'
session_ids = [
    #'20230119',
    '20230329',
    '20230330',
    '20230331',
    '20230401',
    '20230402',
    '20230403',
    '20230403',
    #'20230307',
    #'20230309'
]

sessions_early  = [
    '20230127',
    '20230128',
]

#
sessions_late = [
    '20230202',
    '20230203',
]

# conteingency degradation sessions
t = [
    '20230127',
    '20230128'
    ]
cd = [
    '20230130',
    '20230201',    
    ]
r = [
    '20230202',
    '20230203',
    ]

#
early = [
        '20230127',
        '20230128'
        ]
late = [
    '20230202',
    '20230203',
       ]


#
sess = ProcessSession(root_dir,
                      animal_id,
                      session_ids[0])
#
sess.verbose=False
sess.session_ids = session_ids

#
sess.sessions_early = sessions_early
sess.sessions_late = sessions_late

#
sess.pre = t
sess.during = cd
sess.post = r

#
sess.early = early
sess.late = late




In [64]:
############################################################
####### GENERATE REWARD DFF THRESHOLDS BASED ON DAY 0 ######
############################################################

#
def load_day0_thresholds(root_dir, animal_id):
    fname = os.path.join(root_dir, animal_id, 'day0', 'rois_pixels_and_thresholds_day0.npz')

    d = np.load(fname, allow_pickle=True)

    high_threshold = d['high_threshold']
    print ("high_threshold: ", high_threshold)

    #
    roi_f0s = d['roi_f0s']
    cell_ids = d['cell_ids']

    #
    roi_f0s = roi_f0s[cell_ids]


    return high_threshold, roi_f0s


def load_session_traces(root_dir, animal_id, session_id):

    fname = os.path.join(root_dir, animal_id, session_id, 'data',
                         'results.npz')

    d = np.load(fname, allow_pickle=True)

    #
    reward_times = d['reward_times'].T[:,1]
    n_rewards = np.where(reward_times>0)[0].shape[0]
    print ("n_rewards given: ", n_rewards)

    #
    rois_traces_smooth1 = d['rois_traces_smooth1']
    rois_traces_smooth2 = d['rois_traces_smooth2']

    rois_traces_raw_ensemble1 = d['rois_traces_raw_ensemble1']
    rois_traces_raw_ensemble2 = d['rois_traces_raw_ensemble2']

    print ("rois_traces_raw_ensemble1: ", rois_traces_raw_ensemble1.shape)
    ensemble1_f0 = np.mean(rois_traces_raw_ensemble1, axis=1)
    ensemble2_f0 = np.mean(rois_traces_raw_ensemble2, axis=1)
    print ("ensemble1_f0: ", ensemble1_f0)


    plt.figure()
    plt.plot(rois_traces_raw_ensemble1[0])
    plt.show()

    ensemble_activity[:,:300] = 0

    ensemble_sum = np.sum(ensemble_activity, axis=0)



    return ensemble_sum, n_rewards

def compute_rewards(ensemble, threshold):

    #
    rewards = 0
    ctr = 0
    post_reward = False
    while ctr< ensemble.shape[0]:
        if ensemble[ctr]<(threshold*0.5):
            post_reward = False

        if ensemble[ctr]>threshold:
            if post_reward==False:
                rewards+=1
                ctr+=30*30
                post_reward = True
        

        ctr+=1
    #print ("rewards: ", rewards)

    return rewards


##########################
plt.figure(figsize=(10,10))
cmap = plt.get_cmap('viridis', 7)
ctr=0
for session_id in session_ids:
    print (session_id)
    high_threshold = load_day0_thresholds(root_dir, animal_id)

    #
    ensemble, n_rewards_given = load_session_traces(root_dir, animal_id, session_id)

    #
    rewards_day0 = compute_rewards(ensemble, high_threshold)

    plt.scatter(n_rewards_given, rewards_day0, c=cmap(ctr), s=100)

    ctr+=1

#
plt.plot([0,100],[0,100], '--', c='grey', linewidth=3)
plt.xlim(0,100)
plt.ylim(0,100)
plt.xlabel("Rewards given in experiment", fontsize=20)
plt.ylabel("Rewards based on day0", fontsize=20)
plt.show()
print ("DONE >..")


20230329
high_threshold:  0.9877430087345819
roi_f0s:  (208,)
cell_ids:  [13 11  4  6]
n_rewards given:  34
rois_traces_raw_ensemble1:  (2, 90000)
ensemble1_f0:  [627.2492058410645 492.12202346903484]


NameError: name 'ensemble_activity' is not defined

In [15]:
######################################################
############### SHOW HIT RATES / SESSION #############
######################################################

#
sess.show_plots=True
sess.hit_rate_per_session()

print ("DONE >..")


loading data for hit rate per session: 100%|██████████| 7/7 [00:00<00:00, 5196.48it/s]
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


DONE >..


/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [16]:
######################################################
########### AVERAGE REWARDS INTRASESSION #############
######################################################

#
sess.n_rewards_intra_session()

#
print ("DONE >..")

DONE >..


In [59]:
######################################################
########### AVERAGE REWARDS INTRASESSION #############
######################################################

#
sess.verbose=False
sess.n_rewards_intra_session_normalized()

#

100%|██████████| 7/7 [00:12<00:00,  1.73s/it]


In [81]:
######################################################
########### EVOLUTION OF CELL-XCORRELOGRAMS ##########
######################################################

#
sess.verbose=False
sess.window = 20*30   # in seconds
sess.correlograms_inter_session()

loading data for hit rate per session: 100%|██████████| 7/7 [00:00<00:00, 71.36it/s]

Length of cc:  4 4
Length of cc:  4 4
Length of cc:  4 4
Length of cc:  4 4
Length of cc:  4 4
Length of cc:  4 4
Length of cc:  4 4


In [64]:
######################################################
############### SHOW N-BURSTS / SESSION ##############
######################################################

#
sess.n_bursts_per_session()

#
print ("DONE >..")

loading data for n_bursts_per_session: 100%|██████████| 7/7 [00:00<00:00, 9084.20it/s]


DONE >..


In [26]:
########################################################
########### CONTINGENCY DEGRADATION PLOTS ##############
########################################################

#

#
sess.contingency_degradation()

#


In [31]:
########################################################
########### EARLY VS. LATE PER 5 MIN BLOCK #############
########################################################


#
sess.early_vs_late()

#

/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [32]:
#############################################################
########### EARLY VS. LATE PLOTS - PER SESSION ##############
#############################################################
#
sess.early_vs_late_session()


In [57]:
#############################################################
########### TIME TO REWARD EARLY VS. LATE ###################
#############################################################

#
sess.time_to_reward_per_trial_inter_session_late_vs_early()



[11.2159919  15.75997819]


In [80]:
###########################################
####### BINNED PAIRWISE CORRELATIONS ######
###########################################

#
#sess.session_id = '20230202'
#sess.session_id = '20230131'
sess.pairwise_correlations_inter_session_dff()

loading data for n_bursts_per_session: 100%|██████████| 7/7 [00:00<00:00, 1736.57it/s]


In [ ]:
'/media/cat/4TB/donato/andres/DON-011733/20230127/pairwise_correlations_intra_session_upphase.npz'
'/media/cat/4TB/donato/andres/DON-011733/20230127/results/pairwise_correlations_intra_session_upphase.npz'

